In [65]:
import pandas as pd
import numpy as np
import re
import nltk
import sys
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.regularizers import l2
from nltk.stem import PorterStemmer

#import statments 

In [17]:
data = pd.read_csv('/Users/davidan/Downloads/fade-net/data/data_shuffled.csv')
data['bodytext'] = data['bodytext'].str.lower()

def identify_tokens(row):
    news = row['bodytext']
    tokens = nltk.word_tokenize(news)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

data['words'] = data.apply(identify_tokens, axis=1)

stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

data['stem_meaningful'] = data.apply(remove_stops, axis=1)

def rejoin_words(row):
    my_list = row['stem_meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

data['processed'] = data.apply(rejoin_words, axis=1)

# cols_to_drop = ['words', 'stem_meaningful']
# data.drop(cols_to_drop, inplace=True)

data['bodytext'] = data['processed']


data.head()

,bodytext,label,words,stem_meaningful,processed
0,guardian survey found pervasive culture staff ...,0,"[a, guardian, survey, has, found, that, there,...","[guardian, survey, found, pervasive, culture, ...",guardian survey found pervasive culture staff ...
1,store banning police launched investigation ga...,1,"[store, how, about, banning, all, police, have...","[store, banning, police, launched, investigati...",store banning police launched investigation ga...
2,defended predecessor trade leading delegation ...,0,"[has, defended, his, predecessor, as, trade, f...","[defended, predecessor, trade, leading, delega...",defended predecessor trade leading delegation ...
3,email print jay z performed free get vote conc...,1,"[email, print, and, jay, z, performed, for, fr...","[email, print, jay, z, performed, free, get, v...",email print jay z performed free get vote conc...
4,today given lifetime opportunity tell trump go...,1,"[by, are, today, being, given, a, once, in, a,...","[today, given, lifetime, opportunity, tell, tr...",today given lifetime opportunity tell trump go...


In [21]:
dropcol = ['words', 'stem_meaningful', 'processed']
data.drop(labels = dropcol, axis = 1)

data.to_csv('/Users/davidan/Downloads/fade-net/data/tokenizedData.csv', index = False)

#manually check

In [ ]:
data = pd.read_csv('/Users/davidan/Downloads/fade-net/data/tokenizedData.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data.head()

In [53]:
data = pd.read_csv('/Users/davidan/Downloads/fade-net/data/tokenizedData.csv')


news = data['bodytext'].values
label = data['label'].values

#delete the header file 
news = np.delete(news, (4396), axis=0)
label = np.delete(label, (4396), axis=0)

counter_news = 0

for entry in news:
    news[counter_news] = news[counter_news][:244]

label = np.array(label)

label = label.astype(np.float)



print(label)
print(type(label[2]))

[0. 1. 0. ... 0. 1. 1.]
<class 'numpy.float64'>


In [54]:
print(news[0])

guardian survey found pervasive culture staff bullying parts detrimental impact staff wellbeing patient stories professionals submitted survey guardian diagnosed acute stress incident critically high blood frightened one man thought nothing sex


In [55]:
t = Tokenizer()
t.fit_on_texts(news)

text_matrix = t.texts_to_sequences(news)

print('News: ' + news[0])
print('Representation: ')
print(text_matrix[0])

#find max length of vectors

max_length = 0
min_length = 1
total_length = 0

emb_dim = 16

vocab_size = len(t.word_index) + 1

for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    if max_length < sent_length:
        max_length = sent_length
        
for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    if min_length > sent_length:
        min_length = sent_length

for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    total_length += sent_length

print('Average Length %d' % int(total_length/19255))
print('vocabulary size: %d'%vocab_size)
print('Max Length: %d' % max_length)
print('Min Length: %d' % min_length)

print(type(text_matrix[1]))

News: guardian survey found pervasive culture staff bullying parts detrimental impact staff wellbeing patient stories professionals submitted survey guardian diagnosed acute stress incident critically high blood frightened one man thought nothing sex
Representation: 
[413, 2105, 92, 8134, 648, 444, 5276, 991, 9492, 532, 444, 5702, 2753, 600, 3014, 4103, 2105, 413, 4981, 5787, 2292, 1315, 7115, 146, 1053, 6897, 2, 97, 228, 188, 911]
Average Length 244
vocabulary size: 48543
Max Length: 2524
Min Length: 1
<class 'list'>


In [49]:
print(text_matrix[0])

[413, 2105, 92, 8134, 648, 444]


In [57]:
from tensorflow.keras.regularizers import l2



#split datasets 
#No need to post pad data since all have lengths 
text_matrix = pad_sequences(text_matrix, maxlen=max_length, padding='post')

# text_matrix = np.array(text_matrix, dtype=None)
# label = np.array(label, dtype= None)

X_train, X_test, y_train, y_test = train_test_split(text_matrix, label, test_size = 0.33, random_state = 53)

X_train = tf.ragged.constant(X_train)
X_test = tf.ragged.constant(X_test)
y_train = tf.ragged.constant(y_train)
y_test = tf.ragged.constant(y_test)

lstm_units = 12
inputs = Input(shape = (64,))
embedding = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=244, embeddings_regularizer=l2(.001))
embd_out = embedding(inputs)

lstm = LSTM(lstm_units, dropout = 0.25, recurrent_dropout = 0.2)
lstm_out = lstm(embd_out)

prob = Dense(1, activation = 'relu')
outputs = prob(lstm_out)

model = Model(inputs, outputs)

print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 64, 16)            776688    
_________________________________________________________________
lstm_3 (LSTM)                (None, 12)                1392      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 13        
Total params: 778,093
Trainable params: 778,093
Non-trainable params: 0
_________________________________________________________________
None


In [59]:

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(x=X_train,y=y_train,
          batch_size=80,
          epochs=10,
          verbose=1,
          shuffle=True,
          validation_data=(X_test,y_test)
         )

Epoch 1/10
164/164 [==============================] - 508s 3s/step - loss: 7.6728 - acc: 0.5058 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 2/10
164/164 [==============================] - 440s 3s/step - loss: 7.7659 - acc: 0.4965 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 3/10
164/164 [==============================] - 402s 2s/step - loss: 7.6853 - acc: 0.5018 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 4/10
164/164 [==============================] - 405s 2s/step - loss: 7.7203 - acc: 0.4995 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 5/10
164/164 [==============================] - 415s 3s/step - loss: 7.7882 - acc: 0.4951 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 6/10
164/164 [==============================] - 402s 2s/step - loss: 7.6755 - acc: 0.5024 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 7/10
164/164 [==============================] - 375s 2s/step - loss: 7.7031 - acc: 0.5006 - val_loss: 7.6429 - val_acc: 0.5045
Epoch 8/10
164/164 [==============================] - 370s 2s/step - 

In [68]:
model_3.save('model_3')

NameError: name 'model_3' is not defined